In [1]:
import numpy as np 
import pandas as pd 
import re
from random import randint
from sklearn.model_selection import train_test_split
import tensorflow as tf

batchSize = 1000
lstmUnits = 64
numClasses = 2
iterations = 100000
maxSeqLength = 200
numDimensions=100

training_epochs = 200
display_step = 1

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
def loadGloveModel():
    print "Loading Glove Model"
    f = open('glove.6B.100d.txt','r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print "Done.",len(model)," words loaded!"
    return model

In [6]:
# glove word-vec dictionary
glove = loadGloveModel()

Loading Glove Model
Done. 400000  words loaded!


In [11]:
glove_vecs = pd.Series(glove)
glove_vecs = np.array(list(glove_vecs.values))
print glove_vecs.shape 

glove_keys=np.array(glove.keys(), dtype=object)
print glove_keys.shape
glove_keys = unicode(glove_keys.tolist())

(400000, 100)
(400000,)


In [12]:
matrix_data = np.load('my_train_matrix.npy')
np.random.shuffle(matrix_data)
labels = matrix_data[:-1,-1]
data = matrix_data[:-1,:-1]
print data.shape
print labels.shape

(42999, 200)
(42999,)


In [13]:
def word2vec(word):
    try:
        return glove[word]
    except KeyError:
        return np.zeros((100))

vec_data = [[word2vec(w) for w in story] for story in data]
vec_data = np.array(vec_data)
print vec_data.shape

(42999, 200, 100)


In [14]:
def to_binary(X):
    if X == "REAL":
        return 1
    else:
        return 0

def one_hot_encode(labels):
    labels = map(to_binary, labels)
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), [int(l) for l in labels]] = 1
    return one_hot_encode

binary_labels = one_hot_encode(labels)

In [8]:
x_train, x_val, y_train, y_val = train_test_split(vec_data, binary_labels, test_size=0.20, random_state=42)

In [17]:
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [None, numClasses])
#input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.placeholder(tf.float32, shape=[None, maxSeqLength, numDimensions], name='data')
#tf.Variable(tf.zeros([None, maxSeqLength, numDimensions]),dtype=tf.float32)
#data = tf.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

Instructions for updating:
Use the retry module or similar alternatives.


/anaconda2/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [33]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    new_cost = 0.0
    total_batch = int(len(x_train) / batchSize)
    x_batches = np.array_split(x_train, total_batch)
    y_batches = np.array_split(y_train, total_batch)
    for i in range(total_batch):
        batch_x, batch_y = x_batches[i], y_batches[i]
        _, c = sess.run([optimizer, loss], 
                        feed_dict={data: batch_x, labels: batch_y})
        new_cost = c
    if epoch % display_step == 0:
        print("Epoch:", '%04d' % (epoch+1), "cost=", \
            "{:.9f}".format(new_cost))
print("Optimization Finished!")

('Epoch:', '0001', 'cost=', '0.522030234')
('Epoch:', '0002', 'cost=', '0.402257770')
('Epoch:', '0003', 'cost=', '0.377179325')
('Epoch:', '0004', 'cost=', '0.334135115')
('Epoch:', '0005', 'cost=', '0.317747355')
('Epoch:', '0006', 'cost=', '0.321579754')
('Epoch:', '0007', 'cost=', '0.302445203')
('Epoch:', '0008', 'cost=', '0.274533570')
('Epoch:', '0009', 'cost=', '0.309226215')
('Epoch:', '0010', 'cost=', '0.426435262')
('Epoch:', '0011', 'cost=', '0.334859729')
('Epoch:', '0012', 'cost=', '0.303183258')
('Epoch:', '0013', 'cost=', '0.282682955')
('Epoch:', '0014', 'cost=', '0.321835011')
('Epoch:', '0015', 'cost=', '0.263484567')
('Epoch:', '0016', 'cost=', '0.252099305')
('Epoch:', '0017', 'cost=', '0.435501844')
('Epoch:', '0018', 'cost=', '0.313765854')
('Epoch:', '0019', 'cost=', '0.283792138')
('Epoch:', '0020', 'cost=', '0.245909587')
('Epoch:', '0021', 'cost=', '0.235944539')
('Epoch:', '0022', 'cost=', '0.216612861')
('Epoch:', '0023', 'cost=', '0.216807202')
('Epoch:', 

('Epoch:', '0192', 'cost=', '0.015939135')
('Epoch:', '0193', 'cost=', '0.025663735')
('Epoch:', '0194', 'cost=', '0.020660438')
('Epoch:', '0195', 'cost=', '0.006802618')
('Epoch:', '0196', 'cost=', '0.008750542')
('Epoch:', '0197', 'cost=', '0.075642988')
('Epoch:', '0198', 'cost=', '0.020174643')
('Epoch:', '0199', 'cost=', '0.031561803')
('Epoch:', '0200', 'cost=', '0.042696152')
('Epoch:', '0201', 'cost=', '0.045962743')
('Epoch:', '0202', 'cost=', '0.029400622')
('Epoch:', '0203', 'cost=', '0.018198753')
('Epoch:', '0204', 'cost=', '0.007645612')
('Epoch:', '0205', 'cost=', '0.003228384')
('Epoch:', '0206', 'cost=', '0.002973609')
('Epoch:', '0207', 'cost=', '0.001709450')
('Epoch:', '0208', 'cost=', '0.001554550')
('Epoch:', '0209', 'cost=', '0.002011007')
('Epoch:', '0210', 'cost=', '0.002087251')
('Epoch:', '0211', 'cost=', '0.001472548')
('Epoch:', '0212', 'cost=', '0.003710862')
('Epoch:', '0213', 'cost=', '0.000432693')
('Epoch:', '0214', 'cost=', '0.000960654')
('Epoch:', 

KeyboardInterrupt: 

In [18]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models'))

INFO:tensorflow:Restoring parameters from models/model2.ckpt


In [19]:
test_batches = int(len(x_train) / batchSize)
x_batches = np.array_split(x_val, test_batches)
y_batches = np.array_split(y_val, test_batches)
for i in range(test_batches):
    batch_x, batch_y = x_batches[i], y_batches[i]
    acc = sess.run(accuracy, 
                    feed_dict={data: batch_x, labels: batch_y})
    print("Accuracy for this batch:", acc * 100)

NameError: name 'x_train' is not defined

In [20]:
test_df = pd.read_csv('test.csv')
print test_df.columns
test_text = test_df['text'].values
test_labels = test_df['labels'].values
numTest = test_text.shape[0]

Index([u'text', u'labels'], dtype='object')


In [21]:
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

In [22]:
# stories x the first 200 words
test_matrix = np.empty((numTest, maxSeqLength), dtype='object')
story_index = 0
for story in test_text:
    words = np.empty((maxSeqLength), dtype='object')
    indexCounter = 0
    cleanedLine = cleanSentences(story)
    split = cleanedLine.split()
    if len(split)< maxSeqLength:
        indexCounter = maxSeqLength-len(split)
    else:
        split = split[:maxSeqLength]
    for word in split:
        words[indexCounter] = word
        indexCounter = indexCounter + 1
    test_matrix[story_index] = words
    story_index+=1

In [23]:
test_data = [[word2vec(w) for w in story] for story in test_matrix]
test_data = np.array(test_data)
print test_data.shape
binary_labels = one_hot_encode(test_labels)

(39, 200, 100)


In [25]:
for datum in range(len(test_data)):
    pred = sess.run(prediction, feed_dict={data: test_data[datum].reshape(1,200,100), labels: binary_labels[datum].reshape(1,2)})
    ind = tf.argmax(pred,1).eval()
    label = ''
    if ind[0] == 0:
        label = 'FAKE'
    else:
        label = "REAL"
    print("Prediction:", label, 'Actual:', test_labels[datum])

('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Prediction:', 'FAKE', 'Actual:', 'FAKE')
('Predictio

In [77]:
#np.save('test_data', test_data)
#np.save('test_binary_labels', binary_labels)